In [1]:
import tskit
import dendropy
import glob
import numpy as np
import pandas as pd
import warnings
import functools
import argparse
import operator
import os
from helper_functions import *
%load_ext memory_profiler

In [2]:
parser = argparse.ArgumentParser(description='Gets stats from unioned tree sequence')
parser.add_argument('rand_id', type=str)
parser.add_argument('rep', type=str)
parser.add_argument('win_size', type=lambda x: int(float(x)))
parser.add_argument('sample_size', type=int)
parser.add_argument('coords_dict', type=str, help="String of a dictionary with padded and non-padded start and ends of the chromosomic region. Assumes one chromosome only!")
parser.add_argument('--seed', type=int, default=8991, required=False)

_StoreAction(option_strings=['--seed'], dest='seed', nargs=None, const=None, default=8991, type=<class 'int'>, choices=None, help=None, metavar=None)

In [3]:
%%memit
args = vars(parser.parse_args(["GDVOP9EMEV6ZLEG","0","1000000","10","{'chr': 'chr12', 'start': 60000000, 'end': 70000000, 'padded_start': 60000000, 'padded_end': 70000000}"]))
coords_dict = eval(args['coords_dict'])
out_path = "../../output/"
rng = np.random.default_rng(args['seed'])

peak memory: 93.00 MiB, increment: 0.45 MiB


In [4]:
recap_mut_path = f"{out_path}{args['rand_id']}/{args['rand_id']}_rep{args['rep']}.union.recap.mut.trees"
pops_path = f"{out_path}{args['rand_id']}/{args['rand_id']}_rep{args['rep']}.pops"
with open(pops_path, "r") as f:
    pops = eval(f.readline())

In [5]:
%%memit
assert os.path.exists(recap_mut_path) and os.path.exists(pops_path), f"Trees file or .pops file does not exist for {args['rand_id']}_{args['rep']}"
recap_tsu = tskit.load(recap_mut_path)

peak memory: 8373.65 MiB, increment: 8280.24 MiB


In [ ]:
%%memit
samples = sample_from_ts(recap_tsu, sample_size=args["sample_size"], rng=rng)

In [ ]:
sample_sets = list(samples.values())

In [ ]:
n = np.array([len(s) for s in sample_sets], dtype='float')
num_pops = len(sample_sets)

twoway = [[x,y] for x in range(num_pops) for y in range(num_pops) if x>=y]
fourway = np.array([(twoway[xx] + twoway[yy]) for xx in range(len(twoway)) for yy in range(len(twoway)) if xx>=yy], dtype='int')

i, j, k, l = [fourway[:,x] for x in range(4)]


In [ ]:
def f(x):
    numer = x[i] * (n[j] - x[j]) * x[k] * (n[k] - x[k])
    denom = n[i] * (n[j] - (i == j)) * n[k] * (n[l] - (k == l))
    return numer / denom

In [ ]:
%%memit
covs = recap_tsu.sample_count_stat(sample_sets=sample_sets, f=f, output_dim=fourway.shape[0])

In [ ]:
fourway[1300]

In [ ]:
fourway

In [ ]:
np.where(np.all(fourway == [1,1,0,0], axis=1)==True)

In [ ]:
covs[0]

In [ ]:
covs[1]

In [ ]:
covs[2]

In [ ]:
fourway[3]

In [ ]:
covs[3]

In [ ]:
np.where(np.all(fourway == [7,7,6,6], axis=1)==True)

In [ ]:
covs[657]

In [ ]:
np.where(np.all(fourway == [4,4,0,0], axis=1)==True)

In [ ]:
covs[105]

In [ ]:
np.where(np.all(fourway == [5,5,4,4], axis=1)==True)

In [ ]:
covs[224]

In [ ]:
pops

In [ ]:
import collections


In [ ]:
rank_counts = collections.Counter(t.rank() for t in tssimp.trees())

In [ ]:
most_common = rank_counts.most_common(100)

In [ ]:
most_common[0][0]

In [ ]:
trees = [tskit.Tree.unrank(tssimp.num_samples, mc[0]) for mc in most_common]

In [ ]:
tssimp.num_trees

In [ ]:
from IPython.display import SVG, display, HTML
for i in range(10):
    print(most_common[i])
    display(SVG(trees[i].draw_svg(node_labels = node_labels)))

In [ ]:
multi_tree_str = ""

In [ ]:
for tree in trees:
    multi_tree_str += tree.as_newick(include_branch_lengths=False, node_labels = node_labels)
    multi_tree_str += "\n"

In [ ]:
multi_tree_str

In [ ]:
import toytree

In [ ]:
mtre0 = toytree.mtree(multi_tree_str)

In [ ]:
canvas, axes, mark = mtre0.draw_cloud_tree(
    edge_style={
        "stroke-opacity": 0.1,
        "stroke-width": 1,
    },
);

In [ ]:
# pull out the trees and get a densiplot

In [ ]:
# plot the residuals of those correlations against branch lengths/N or sth? that would help us there.